In [51]:
import pandas as pd
import os, requests, zipfile, warnings, time
from io import BytesIO
import datetime
warnings.filterwarnings('ignore')

In [30]:
test_url = 'http://data.gdeltproject.org/gdeltv2/20250219000000.gkg.csv.zip'
base_url = 'http://data.gdeltproject.org/gdeltv2/'

In [ ]:
def parse_data(data):
    df = pd.read_csv(
        data,
        delimiter='\t',
        encoding='utf-8',
        low_memory=False,
        header=None,
        usecols=[0, 1, 2, 3, 4, 7, 9, 13, 15, 17, 25, 26],
        names=[
            'GKGRECORDID',
            'V2.1DATE',
            'V2SOURCECOLLECTIONIDENTIFIER',
            'V2SOURCECOMMONNAME',
            'V2DOCUMENTIDENTIFIER',
            'V1THEMES',
            'V1LOCATIONS',
            'V1ORGANIZATIONS',
            'V1.5TONE',
            'V2GCAM',
            'V2.1TRANSLATIONINFO',
            'V2EXTRASXML'
        ]
    )
    return(df)

In [35]:
def download_unzip(url):
    response = requests.get(url)

    # Create a BytesIO object from the downloaded content
    zip_bytes = BytesIO(response.content)

    # Open the zip file in memory
    with zipfile.ZipFile(zip_bytes, 'r') as zip_file:
        # Get the first file in the zip
        first_file = zip_file.namelist()[0]
        
        # Read the file content directly into memory
        with zip_file.open(first_file) as file:
            return(parse_data(file))

In [36]:
download_unzip(test_url)

,GKGRECORDID,V2.1DATE,V2SOURCECOLLECTIONIDENTIFIER,V2SOURCECOMMONNAME,V2DOCUMENTIDENTIFIER,V1THEMES,V1LOCATIONS,V1ORGANIZATIONS,V1.5TONE,V2GCAM,V2.1TRANSLATIONINFO,V2EXTRASXML
0,20250219000000-0,20250219000000,1,mmm-online.com,https://www.mmm-online.com/features/ai-encroac...,TAX_FNCACT;TAX_FNCACT_CMO;TAX_ECON_PRICE;WB_62...,1#France#FR#FR#46#2#FR;1#Canada#CA#CA#60#-96#CA,pfizer;bristol myers squibb;martin agency;amge...,"1.02657004830918,2.89855072463768,1.8719806763...","wc:1503,c1.2:3,c12.1:122,c12.10:143,c12.12:46,...",NaN,<PAGE_LINKS>https://www.interpublic.com/news/s...
1,20250219000000-1,20250219000000,1,nbcrightnow.com,https://www.nbcrightnow.com/news/crystal-apple...,EDUCATION;TAX_FNCACT;TAX_FNCACT_EDUCATORS;WB_4...,NaN,finley elementary school;desert oasis high sch...,"9.41176470588235,9.41176470588235,0,9.41176470...","wc:151,c12.1:11,c12.10:6,c12.13:2,c12.14:4,c12...",NaN,<PAGE_AUTHORS>Thomas Metcalf;NonStop Local Dig...
2,20250219000000-2,20250219000000,1,centralwesterndaily.com.au,https://www.centralwesterndaily.com.au/story/8...,KILL;CRISISLEX_T03_DEAD;DISASTER_FIRE;CRISISLE...,"4#Sydney, New South Wales, Australia#AS#AS02#-...",australian associated,"-2.87610619469027,1.10619469026549,3.982300884...","wc:410,c12.1:23,c12.10:31,c12.12:16,c12.13:7,c...",NaN,<PAGE_PRECISEPUBTIMESTAMP>20250218225100</PAGE...
3,20250219000000-3,20250219000000,1,wlfi.com,https://www.wlfi.com/news/ive-got-many-levers-...,LEADER;TAX_FNCACT;TAX_FNCACT_GOVERNOR;ELECTION...,"3#Terre Haute, Indiana, United States#US#USIN#...",braun,"-0.681431005110733,2.72572402044293,3.40715502...","wc:557,c12.1:34,c12.10:62,c12.12:20,c12.13:22,...",NaN,<PAGE_AUTHORS>Ben Verbanic;By: Ben Verbanic</P...
4,20250219000000-4,20250219000000,1,ksat.com,https://www.ksat.com/news/politics/2025/02/18/...,TAX_WORLDLANGUAGES;TAX_WORLDLANGUAGES_KANSAS;B...,"2#Kansas, United States#US#USKS#38.5111#-96.80...",NaN,"-3.96039603960396,2.64026402640264,6.600660066...","wc:273,c12.1:28,c12.10:31,c12.12:9,c12.13:5,c1...",NaN,<PAGE_LINKS>https://apnews.com/article/transge...
...,...,...,...,...,...,...,...,...,...,...,...,...
1827,20250219000000-1827,20250219000000,1,dailyherald.com,https://www.dailyherald.com/20250218/crime/man...,TAX_FNCACT;TAX_FNCACT_MAN;TRIAL;KILL;CRIME_ILL...,NaN,NaN,"-9.40594059405941,1.48514851485149,10.89108910...","wc:174,c12.1:16,c12.10:14,c12.12:7,c12.13:4,c1...",NaN,<PAGE_AUTHORS>Susan Sarkauskas</PAGE_AUTHORS><...
1828,20250219000000-1828,20250219000000,1,theboxhouston.com,https://theboxhouston.com/11602732/dangelo-mee...,TAX_FNCACT;TAX_FNCACT_SINGER;CRISISLEX_C07_SAF...,"3#Philadelphia, Pennsylvania, United States#US...",mann center,"4.4,5.6,1.2,6.8,21.2,0.4,216","wc:216,c1.2:1,c12.1:17,c12.10:18,c12.12:3,c12....",NaN,<PAGE_LINKS>https://therootspicnic.com/;https:...
1829,20250219000000-1829,20250219000000,1,pr-inside.com,https://www.pr-inside.com/contact-levi-korsins...,ALLIANCE;LEGISLATION;WB_845_LEGAL_AND_REGULATO...,1#United States#US#US#39.828175#-98.5795#US,walgreens boots alliance inc;news network;walg...,"-4.32801822323462,2.27790432801822,6.605922551...","wc:348,c1.3:1,c12.1:22,c12.10:43,c12.12:11,c12...",NaN,<PAGE_LINKS>http://Esq.33;http://Esq.Ed;http:/...
1830,20250219000000-1830,20250219000000,1,ncpa.org,https://www.ncpa.org/newsroom/qam/2025/02/19/s...,LEADER;TAX_FNCACT;TAX_FNCACT_PRESIDENT;USPEC_P...,1#Japan#JA#JA#36#138#JA;1#United States#US#US#...,national archives;supreme court,"-5.18518518518519,1.48148148148148,6.666666666...","wc:247,c12.1:15,c12.10:27,c12.12:12,c12.13:11,...",NaN,<PAGE_LINKS>https://www.archives.gov/education...


In [45]:
def generate_url_list(start_dt, end_dt, increment_minutes=15):
    url_list = []
    current_dt = start_dt
    
    while current_dt <= end_dt:
        # Format as YYYYMMDDHHMMSS
        datetime_str = current_dt.strftime('%Y%m%d%H%M%S')
        url_list.append(f'{base_url}{datetime_str}.gkg.csv.zip')
        
        # Add increment
        current_dt += datetime.timedelta(minutes=increment_minutes)
    
    return url_list

In [47]:
start = datetime.datetime(2018, 1, 1, 0, 0, 0)
end = datetime.datetime(2025, 5, 1, 0, 0, 0)
urls = generate_url_list(start, end, 15)
print(urls[0:10])

['http://data.gdeltproject.org/gdeltv2/20180101000000.gkg.csv.zip', 'http://data.gdeltproject.org/gdeltv2/20180101001500.gkg.csv.zip', 'http://data.gdeltproject.org/gdeltv2/20180101003000.gkg.csv.zip', 'http://data.gdeltproject.org/gdeltv2/20180101004500.gkg.csv.zip', 'http://data.gdeltproject.org/gdeltv2/20180101010000.gkg.csv.zip', 'http://data.gdeltproject.org/gdeltv2/20180101011500.gkg.csv.zip', 'http://data.gdeltproject.org/gdeltv2/20180101013000.gkg.csv.zip', 'http://data.gdeltproject.org/gdeltv2/20180101014500.gkg.csv.zip', 'http://data.gdeltproject.org/gdeltv2/20180101020000.gkg.csv.zip', 'http://data.gdeltproject.org/gdeltv2/20180101021500.gkg.csv.zip']


In [48]:
# function to chunk datetime_strings into groups of 100
def chunk_datetime_strings(urls, chunk_size=100):
    return [urls[i:i + chunk_size] for i in range(0, len(urls), chunk_size)]
chunked_datetime_strings = chunk_datetime_strings(urls, 100)

In [ ]:
def download_chunk(chunk):
    # List to store successfully downloaded DataFrames
    df_list = []
    # Create log file name with current datetime
    current_time = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    log_file = f"failed_downloads_{current_time}.txt"
    
    for url in chunk:
        try:
            # Try to download and unzip the file, then parse it into a DataFrame
            df = download_unzip(url)
            df_list.append(df)
        except Exception as e:
            # If download or parsing fails, log the URL and error message
            with open(log_file, "a") as f:
                f.write(f"{url}\t{e}\n")
            print(f"Failed to download or parse {url}: {e}")

        time.sleep(1)  # Sleep to avoid overwhelming the server   
    if df_list:
        # Concatenate all DataFrames if any were successfully downloaded
        return pd.concat(df_list, ignore_index=True)
    else:
        # If none succeeded, print a message and return None
        print(f"None of the URLs worked.")
        return None

In [50]:
test=urls[0:3]
df=download_chunk(test)
df

,GKGRECORDID,V2.1DATE,V2SOURCECOLLECTIONIDENTIFIER,V2SOURCECOMMONNAME,V2DOCUMENTIDENTIFIER,V1THEMES,V1LOCATIONS,V1ORGANIZATIONS,V1.5TONE,V2GCAM,V2.1TRANSLATIONINFO,V2EXTRASXML
0,20180101000000-0,20180101000000,1,stamfordadvocate.com,http://www.stamfordadvocate.com/news/politics/...,TAX_FNCACT;TAX_FNCACT_CHIEF;WB_696_PUBLIC_SECT...,"2#Florida, United States#US#USFL#27.8333#-81.7...",NaN,"-3.64583333333333,3.38541666666667,7.03125,10....","wc:350,c12.1:42,c12.10:33,c12.12:8,c12.13:13,c...",NaN,<PAGE_PRECISEPUBTIMESTAMP>20171231231600</PAGE...
1,20180101000000-1,20180101000000,1,thestar.com.my,https://www.thestar.com.my/business/business-n...,NaN,1#Malaysia#MY#MY#2.5#112.5#MY;1#Japan#JA#JA#36...,public investment bank;zhejiang geely holding ...,"1.51098901098901,2.47252747252747,0.9615384615...","wc:671,c1.2:9,c1.3:1,c12.1:18,c12.10:59,c12.11...",NaN,<PAGE_LINKS>https://www.thestar.com.my/Search<...
2,20180101000000-2,20180101000000,1,radaronline.com,https://radaronline.com/celebrity-news/bow-wow...,TAX_ETHNICITY;TAX_ETHNICITY_AMERICAN;TAX_WORLD...,1#United States#US#US#39.828175#-98.5795#US,NaN,"4.66101694915254,5.50847457627119,0.8474576271...","wc:206,c12.1:31,c12.10:21,c12.12:6,c12.13:8,c1...",NaN,<PAGE_LINKS>http://radaronline.com/tag/bow-wow...
3,20180101000000-3,20180101000000,1,sanantoniopost.com,http://www.sanantoniopost.com/news/255927392/g...,NaN,"3#Houston, Texas, United States#US#USTX#29.763...",NaN,"-0.704225352112676,1.93661971830986,2.64084507...","wc:527,c12.1:24,c12.10:43,c12.12:19,c12.13:7,c...",NaN,<PAGE_LINKS>http://www.sanantoniopost.com/revi...
4,20180101000000-4,20180101000000,1,complex.com,http://www.complex.com/life/2017/12/dinosaur-e...,NaN,"1#China#CH#CH#35#105#CH;4#Ganzhou, Jiangxi, Ch...",NaN,"-0.495049504950495,1.48514851485149,1.98019801...","wc:187,c1.3:1,c12.1:9,c12.10:10,c12.12:4,c12.1...",NaN,<PAGE_LINKS>http://www.cnn.com/2015/04/21/asia...
...,...,...,...,...,...,...,...,...,...,...,...,...
4068,20180101003000-1509,20180101003000,1,abc22now.com,http://abc22now.com/news/entertainment/foundat...,TAX_FNCACT;TAX_FNCACT_OFFICIALS;CRISISLEX_T11_...,"2#California, United States#US#USCA#36.17#-119...",hudson-webber foundation;barbara erb family fo...,"4.19161676646707,4.79041916167665,0.5988023952...","wc:148,c12.1:8,c12.10:8,c12.12:2,c12.13:2,c12....",NaN,<PAGE_AUTHORS>The Associated Press;https://www...
4069,20180101003000-1510,20180101003000,1,steamboattoday.com,http://www.steamboattoday.com/news/make-way-fo...,NATURAL_DISASTER;NATURAL_DISASTER_FLOODS;TAX_W...,"2#New York, United States#US#USNY#42.1497#-74....",new york times,"-0.136612021857923,3.14207650273224,3.27868852...","wc:629,c12.1:45,c12.10:63,c12.12:20,c12.13:24,...",NaN,<PAGE_LINKS>https://twitter.com/audrey_dwyer1<...
4070,20180101003000-1511,20180101003000,1,iheart.com,https://easy1350.iheart.com/content/2017-12-31...,MANMADE_DISASTER_IMPLIED;ROAD_INCIDENT;MANMADE...,"2#Wisconsin, United States#US#USWI#44.2563#-89...",NaN,"-1.15830115830116,1.93050193050193,3.088803088...","wc:226,c1.1:1,c12.1:21,c12.10:10,c12.12:2,c12....",NaN,<PAGE_LINKS>http://jessicabrandau.com;http://t...
4071,20180101003000-1512,20180101003000,1,ledgergazette.com,https://ledgergazette.com/2017/12/31/callon-pe...,ECON_STOCKMARKET;TAX_ECON_PRICE;ENV_OIL;ENV_NA...,1#United States#US#US#39.828175#-98.5795#US;1#...,callon petroleum company;pitcairn co;renaissan...,"1.50462962962963,2.19907407407407,0.6944444444...","wc:779,c1.2:4,c12.1:27,c12.10:38,c12.12:6,c12....",NaN,<PAGE_LINKS>;https://ledgergazette.com/daily-e...
